In [2]:
import os
import pandas as pd
import configparser
#import mysql.connector
from sqlalchemy import create_engine

import seaborn as sns
import matplotlib.pyplot as plt

config = configparser.ConfigParser()
config.read('..\\config.ini')

host = config['mysql']['host']
database = config['mysql']['database']
user = config['mysql']['user']
password = config['mysql']['password']
port = config['mysql']['port']

In [3]:
def read_query(query):
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    df = pd.read_sql(query, con=engine)
    print('Query Executed!')
    return df

In this notebook we will start by exploring the `transactions` table to get a better idea of the items. Once we have a better idea, we can decide on the type of recommender system we should build.

Checking the basic structure of the data:

In [6]:
query = """
SELECT *
FROM transactions
LIMIT 20;
"""

read_query(query)

Query Executed!


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
5,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2
6,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2
7,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,0.030492,1
8,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,0.053373,1
9,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,0.045746,2


Checking the starting date and end date:

In [4]:
query = """
SELECT 
    MAX(t_dat) AS end_date, MIN(t_dat) AS start_date
FROM
    transactions;
"""

read_query(query)

Query Executed!


,end_date,start_date
0,2020-09-22,2018-09-20


The dataset contains transaction history of customers from 2018-09-18 to 2020-09-22 which is roughly 2 years.

When training our model, this feature will be of no use but instead of using the specific dates, we can extract the month and even check if the day is a weekend or not since this will help us get a better idea of the spending habits of the customers. 

However, before we do these feature engineering, we will need to remove `customer_id` not present in the `customer_preprocessed` table we created earlier. Moreover, the data contains transaction activity of both online and in-person shopping but since we are trying to create a recommender for only online users, we can remove instances where sales_channel_id = 1 (indicating store purchases). 

Checking if price of product remained same over these 2 years:

In [3]:
q = """
SELECT 
    T.article_id, COUNT(*) AS price_changes
FROM
    (SELECT 
        article_id
    FROM
        transactions
    WHERE
        sales_channel_id = 2
    GROUP BY article_id , price) T
GROUP BY T.article_id;
"""

all_changes = read_query(q)
all_changes

Query Executed!


,article_id,price_changes
0,108775015,130
1,108775044,125
2,108775051,11
3,110065001,43
4,110065002,28
...,...,...
98317,952267001,8
98318,952938001,3
98319,953450001,2
98320,953763001,7


We can see clearly that the same article_id has different prices indicating that the price for that item had changed over the years.

Since price is always an important factor when trying to decide what to buy we clearly need to fix this issue. The site where this model will be deployed will have the price of the items always fixed, so we need to do the same for our training set. 

In [5]:
location_of_subfolders = "..//h-and-m-personalized-fashion-recommendations//images"

# Get a list of all files in the folder
subfolder_names = os.listdir(location_of_subfolders)

all_articles = []

# Print the names of all files in the folder
for name in subfolder_names:
    location_of_images = os.path.join(location_of_subfolders,name)
    image_names = os.listdir(location_of_images)
    all_articles = all_articles + [int(x.replace('.jpg', '')) for x in image_names]

In [6]:
all_items = """
SELECT 
    *
FROM
    transactions
WHERE
    sales_channel_id = 2
"""

items_all = read_query(all_items)

Query Executed!


In [7]:
new_articles_df = items_all[items_all['article_id'].isin(all_articles)]

In [ ]:
q ="""
SELECT 
    *
FROM
    transactions
WHERE
    sales_channel_id = 2
"""

## Preprocessing for a Popularity based Rec System

Before we continue, we will first find the top most popular items and then save them to be used for a popularity based Rec Sys.

In [1]:
q = """
SELECT 
    T.article_id, COUNT(*) AS price_changes
FROM
    (SELECT 
        article_id
    FROM
        transactions
    WHERE
        sales_channel_id = 2
    GROUP BY article_id , price) T
GROUP BY T.article_id;
"""

all_changes = read_query(q)
all_changes

NameError: name 'read_query' is not defined